Imports

In [ ]:
import copy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from tabulate import tabulate
from google.colab import drive 
import copy
from scipy.signal import tf2ss, ss2tf
import keras.backend as K 
import pickle
from numpy.linalg import matrix_power


Configuration

In [ ]:
# number of experiments to conduct with same configuration:
NUM_EXPERIMENTS = 3

# If True, no memory, i.e. A=0 and A!=0 otherwise
memory_less = False

configs = [{"RANDOM_NORMAL_INIT": True, "A_INIT_TO_SYMETRIC": False, "INIT_B_EQUAL_C": False, 
    "is_lstm": False, "use_adversary": False, "GRU": False, "seq_len": 5, "hidden_dim": 200, "teacher_dims": [1, 2, 4, 6, 8]},

    {"RANDOM_NORMAL_INIT": True, "A_INIT_TO_SYMETRIC": False, "INIT_B_EQUAL_C": False, 
    "is_lstm": True, "use_adversary": False, "GRU": False, "seq_len": 5, "hidden_dim": 200, "teacher_dims": [1, 2, 4, 6, 8]},
    
    {"RANDOM_NORMAL_INIT": True, "A_INIT_TO_SYMETRIC": False, "INIT_B_EQUAL_C": False, 
    "is_lstm": True, "use_adversary": False, "GRU": True, "seq_len": 5, "hidden_dim": 200, "teacher_dims": [1, 2, 4, 6, 8]}]

Constants

In [ ]:
VARIANCE = 0.01

# Number of epocs:
default_n_epocs_rnn = 500

In [ ]:
def apply_model(params, X):
    A, B, C = params
    seq_len = X.shape[1]
    response_vector = np.zeros((seq_len))
    for i in range(seq_len):
        A_pow_i = matrix_power(A, seq_len-i-1)
        response_vector[i] = np.matmul(C, np.matmul(A_pow_i, B))
    return np.matmul(X, response_vector.reshape(-1,1))

Create regular and adversary parpameter for teacher RNN

In [ ]:
import numpy as np
from numpy.linalg import matrix_power

def construct_memory(d=5, memory_less=False):
    if not memory_less:
        A = np.random.randint(low=1, high=3, size=(d, d))
        A = np.tril(A,-1)
    else:
        A = np.zeros((d, d))
    B = np.random.normal(size=(d, 1))
    C = np.random.normal(size=(1, d))
    
    for i in range(1, 10):
        A_i = matrix_power(A,i)
    return (A,B,C)

In [ ]:
def get_rnn_model():
    model = keras.Sequential()

    if not is_lstm:
        print("creating Linear RNN")
        model.add(layers.SimpleRNN(hidden_dim, 
                    kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE),
                    recurrent_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE),
                    activation=None,
                    recurrent_constraint=None,
                    recurrent_regularizer=None,
                    use_bias=False)
        )
    else:
        if GRU:
            print("creating GRU")
            model.add(layers.GRU(hidden_dim, 
                        kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE),
                        recurrent_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE)))
        else:
            print("creating LSTM")
            model.add(layers.LSTM(hidden_dim, 
                        kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE),
                        recurrent_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE)))

    model.add(layers.Dense(1, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0., stddev=VARIANCE)))
    model.compile(
        loss = keras.losses.MeanSquaredError(),
        optimizer = optimizers.Adam(),
        metrics = ['mse'],
    )
    return model

In [ ]:
def make_train_generator(params, n_examples, seq_len):
    X_train = np.random.normal(scale=VARIANCE, size=(n_examples, seq_len))
    Y_train = apply_model(params, X_train)
    X_train = np.expand_dims(X_train, -1)
    while True:
        yield X_train, Y_train

def combine_generators(gen_list):
    while True:
        for gen in gen_list:
            yield next(gen)

In [ ]:
def handle_linear_rnn():
    n_examples_train = 2000
    n_examples_test = 100

    params = construct_memory(teacher_hidden_state)
    train_generator1 = make_train_generator(params, n_examples_train, curr_seq_len)

    model = get_rnn_model()
    callbacks = []
    model.fit_generator(train_generator1, steps_per_epoch=1, epochs=default_n_epocs_rnn, callbacks=callbacks)

    test_loss_arr = []
    seq_len_test = [6, 7, 8, 9, 10]
    for seq_len in seq_len_test:
        X_test = np.random.normal(scale=VARIANCE, size=(n_examples_test, seq_len))
        Y_test = apply_model(params, X_test)
        X_test = np.expand_dims(X_test, -1)
        results = model.evaluate(X_test, Y_test, batch_size=128)
        test_loss_arr.append(results[0])
        mse_or_accuracy_per_sequence_length = test_loss_arr
    mse_or_accuracy_per_sequence_length = {f'{hidden_dim}': {}}
    for t in range(len(seq_len_test)):
        mse_or_accuracy_per_sequence_length[f'{hidden_dim}'][seq_len_test[t]] = test_loss_arr[t]
    
    return mse_or_accuracy_per_sequence_length

Visualization

In [ ]:
def display_mse_results(mse_or_accuracy_per_sequence_length_per_hidden_dim):
    fig, axs = plt.subplots(1,3)
    fig.set_size_inches(30, 10)

    # title = f"Linear RNN of size {hidden_dim} lerning Teacher Linear RNN on sequence length {curr_seq_len}"
    # fig.suptitle(title)

    ax1=axs[0]
    ax2=axs[1]
    ax3=axs[2]

    ax1.set_xlabel('Teacher dimension', fontsize=30)
    ax2.set_xlabel('Teacher dimension', fontsize=30)
    ax3.set_xlabel('Teacher dimension', fontsize=30)

    ax1.set_ylabel('Extrapolation error', fontsize=30)
    ax2.set_ylabel('Extrapolation error', fontsize=30)
    ax3.set_ylabel('Extrapolation error', fontsize=30)

    mse_aggregated = {}

    for k, d, ml, lstm_or_gru, gru, use_adversary, teacher_hidden_state in mse_or_accuracy_per_sequence_length_per_hidden_dim[0].keys():
        mse_aggregated[k, d, ml, lstm_or_gru, gru, use_adversary, teacher_hidden_state] = []
        for exp in range(NUM_EXPERIMENTS):
            mse_or_accuracy_per_sequence_length = mse_or_accuracy_per_sequence_length_per_hidden_dim[exp][k, d, ml, lstm_or_gru, gru, use_adversary, teacher_hidden_state][f'{d}']
            mse = np.array(list(mse_or_accuracy_per_sequence_length.values()))[:-1]

            extrapolation_error = np.sum(mse) / len(mse)
            mse_aggregated[k, d, ml, lstm_or_gru, gru, use_adversary, teacher_hidden_state].append(extrapolation_error)

    mse_dic = {'linear rnn': [], 'gru': [], 'lstm': []}
    min_dic = {'linear rnn': [], 'gru': [], 'lstm': []}
    max_dic = {'linear rnn': [], 'gru': [], 'lstm': []}

    for k, d, ml, lstm_or_gru, gru, use_adversary, teacher_hidden_state in mse_aggregated.keys():    

        mse_aggregated_specific = mse_aggregated[k, d, ml, lstm_or_gru, gru, use_adversary, teacher_hidden_state]
        
        mse = np.mean(mse_aggregated_specific)    
        min_mse = np.min(mse_aggregated_specific)
        max_mse = np.max(mse_aggregated_specific)

        if not lstm_or_gru:
            key = 'linear rnn'
        else:
            if gru:
                key = 'gru'
            else:
                key = 'lstm'
        mse_dic[key].append(mse)
        min_dic[key].append(min_mse)
        max_dic[key].append(max_mse)


    for key in mse_dic.keys():
        mse_list = np.array(mse_dic[key])
        min_list = np.array(min_dic[key])
        max_list = np.array(max_dic[key])

        if key == 'gru':
            tmp_ax = ax3
            label = 'GRU'
        elif key == 'lstm':
            tmp_ax = ax2
            label = 'LSTM'
        else:
            tmp_ax = ax1
            label = 'Linear RNN'
    
        tmp_ax.errorbar(teacher_dims[:], mse_list[:], [mse_list[:] - min_list[:], max_list[:] - mse_list[:]],
                    fmt='.k', ecolor='red', lw=3, capsize=10, label=label)


    font = {'family' : 'normal',
            'weight' : 'bold',
            'size'   : 20}
    matplotlib.rc('font', **font)


    ax1.legend(loc='upper left', fontsize=20)
    ax2.legend(loc='upper left', fontsize=20)
    ax3.legend(loc='upper left', fontsize=20)

    ax1.grid()
    ax2.grid()
    ax3.grid()

    ax1.set_yscale('log')
    ax2.set_yscale('log')
    ax3.set_yscale('log')


Main function:

In [ ]:
mse_or_accuracy_per_sequence_length_per_hidden_dim = {}

for exp in range(NUM_EXPERIMENTS):
    print(f"we are on exp: {exp}")
    if exp not in mse_or_accuracy_per_sequence_length_per_hidden_dim.keys():
        mse_or_accuracy_per_sequence_length_per_hidden_dim[exp] = {}
    for config in configs:
        RANDOM_NORMAL_INIT = config["RANDOM_NORMAL_INIT"]
        A_INIT_TO_SYMETRIC = config["A_INIT_TO_SYMETRIC"]
        INIT_B_EQUAL_C = config["INIT_B_EQUAL_C"]
        is_lstm = config["is_lstm"]
        use_adversary = config["use_adversary"]
        GRU = config["GRU"]
        curr_seq_len = config["seq_len"]
        hidden_dim = config["hidden_dim"]
        teacher_dims = config['teacher_dims']

        for teacher_hidden_state in teacher_dims:
            print(f"Exp: {exp}, is_lstm: {is_lstm}, GRU: {GRU}, teacher_hidden_state: {teacher_hidden_state}")
            
            range_low_j_high_j = [hidden_dim]
            
            mse_or_accuracy_per_sequence_length = handle_linear_rnn()

            mse_or_accuracy_per_sequence_length_per_hidden_dim[exp][(curr_seq_len, hidden_dim, memory_less, is_lstm, GRU, use_adversary, teacher_hidden_state)] = mse_or_accuracy_per_sequence_length

display_mse_results(mse_or_accuracy_per_sequence_length_per_hidden_dim)

we are on exp: 0
Exp: 0, is_lstm: False, GRU: False, teacher_hidden_state: 1
creating Linear RNN


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Streaming output truncated to the last 5000 lines.
Epoch 211/500
1/1 [==============================] - 0s 76ms/step - loss: 1.1200e-11 - mse: 1.1200e-11
Epoch 212/500
1/1 [==============================] - 0s 69ms/step - loss: 1.0895e-11 - mse: 1.0895e-11
Epoch 213/500
1/1 [==============================] - 0s 80ms/step - loss: 1.0602e-11 - mse: 1.0602e-11
Epoch 214/500
1/1 [==============================] - 0s 78ms/step - loss: 1.0320e-11 - mse: 1.0320e-11
Epoch 215/500
1/1 [==============================] - 0s 82ms/step - loss: 1.0050e-11 - mse: 1.0050e-11
Epoch 216/500
1/1 [==============================] - 0s 71ms/step - loss: 9.7898e-12 - mse: 9.7898e-12
Epoch 217/500
1/1 [==============================] - 0s 76ms/step - loss: 9.5403e-12 - mse: 9.5403e-12
Epoch 218/500
1/1 [==============================] - 0s 81ms/step - loss: 9.3003e-12 - mse: 9.3003e-12
Epoch 219/500
1/1 [==============================] - 0s 78ms/step - loss: 9.0696e-12 - mse: 9.0696e-12
Epoch 220/500
1/1 [===